In [2]:
import requests
import json

# API 调用配置
api_url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
api_key = "f28f99afec54ba87aef362f6a6407d3b.kWHsoEEitX2e2Ro5"  # 替换为您的API密钥

# 请求头
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

# 请求参数
data = {
    "model": "glm-4-plus",  # 模型编码
    "messages": [{"role": "user", "content": "python是什么？"}],  # 对话消息列表
}

# 发送 POST 请求
response = requests.post(api_url, headers=headers, json=data)

# 检查响应状态码
if response.status_code == 200:
    # 处理响应数据
    print(response.json())
else:
    # 处理错误情况
    print(f"Error: {response.status_code} - {response.text}")


{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'Python是一种高级编程语言，由Guido van Rossum于1989年发明，并于1991年首次发布。Python的设计哲学强调代码的可读性和简洁的语法（尤其是使用空格缩进来表示代码块，而不是使用大括号或关键词）。这使得Python成为初学者入门编程的理想选择，同时也适用于专业人士进行复杂系统开发。\n\n### Python的主要特点包括：\n\n1. **易于学习和使用**：Python语法简单明了，接近英语自然语言，容易上手。\n2. **强大的库支持**：Python有大量的标准库和第三方库，涵盖从网络开发、数据分析到人工智能等众多领域。\n3. **跨平台**：Python可以在Windows、macOS、Linux等多种操作系统上运行。\n4. **解释型语言**：Python代码在运行时由解释器逐行解释执行，无需编译。\n5. **开源和免费**：Python是开源软件，可自由使用和分发。\n\n### Python的应用领域：\n\n- **Web开发**：使用Django、Flask等框架进行网站和Web应用开发。\n- **数据分析**：利用Pandas、NumPy等库进行数据分析和处理。\n- **人工智能与机器学习**：通过TensorFlow、PyTorch等库进行AI和机器学习项目的开发。\n- **自动化脚本**：编写脚本自动化日常任务。\n- **教育与研究**：因其易读性，Python常被用于教学和科研。\n\nPython因其广泛的应用和强大的社区支持，已成为当今最流行的编程语言之一。无论是初学者还是有经验的开发者，Python都是一个值得学习和使用的工具。', 'role': 'assistant'}}], 'created': 1732639731, 'id': '202411270048431fb2d69d52914dc8', 'model': 'glm-4-plus', 'request_id': '202411270048431fb2d69d52914dc8', 'usage': {'completion_tokens': 325, 'prompt_tokens': 

In [4]:
pip install langchain -i https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simpleNote: you may need to restart the kernel to use updated packages.



In [7]:
import json
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import Embeddings
from langchain.text_splitter import CharacterTextSplitter
# 自定义智普AI嵌入类
class ZhipuAIEmbeddings(Embeddings):
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        self.api_url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"

    def embed_documents(self, texts):
        # 这里需要根据智普AI的API要求来构造请求
        # 注意：以下代码仅为示例，可能需要根据实际API进行调整
        data = {
            "model": "glm-4-plus",
            "messages": [{"role": "user", "content": text} for text in texts]
        }
        response = requests.post(self.api_url, headers=self.headers, json=data)
        if response.status_code == 200:
            return [item['message']['content'] for item in response.json()['choices']]
        else:
            raise Exception(f"Error from Zhipu AI API: {response.status_code} - {response.text}")

# 读取JSON文档
with open('D:\大一作业\实用机器学习\实训下午\qa_pairs.json', 'r') as file:
    data = json.load(file)
    texts = [doc['text'] for doc in data['documents']]  # 假设JSON结构包含一个'documents'键，每个文档都有一个'text'键

# 文本分割
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(texts)

# 创建向量存储
zhipu_embeddings = ZhipuAIEmbeddings(api_key="f28f99afec54ba87aef362f6a6407d3b.kWHsoEEitX2e2Ro5")
vectorstore = FAISS.from_documents(split_docs, zhipu_embeddings)

# 设置问答链
qa = RetrievalQA.from_chain_type(llm_chain_type="stuff", retriever=vectorstore.as_retriever())

# 进行问答
query = "你的问题是什么？"
result = qa.run(query)
print(result)


ImportError: cannot import name 'Embeddings' from 'langchain.embeddings' (C:\ProgramData\Anaconda3\lib\site-packages\langchain\embeddings\__init__.py)

In [1]:
import os
import json
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# 设置环境变量
os.environ["OPENAI_API_KEY"] = "sk-QrDToYPo9bfTR3S6H27qTJ3NdHcDMXkBZszGOHkgidESUn4h"  # 使用 OpenAI 的嵌入作为示例

# 知识库路径
knowledge_base_path = 'D:\\大一作业\\实用机器学习\\实训下午\\qa_pairs.json'

# 读取 JSON 文件
with open(knowledge_base_path, 'r', encoding='GB2312') as file:
    data = json.load(file)

# 创建 Document 对象列表
documents = [Document(page_content=f"{item['question']}\n{item['answer']}") for item in data]

# 使用 CharacterTextSplitter 分割文档
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(documents)

# 创建向量存储
embeddings = OpenAIEmbeddings()  # 使用 OpenAI 的嵌入作为示例
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 创建检索 QA
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever())

# 生成问答对的函数
def generate_qapair(question: str) -> dict:
    answer = qa.run(question)
    return {"question": question, "answer": answer}

# 示例：生成问答对
question = "LangChain 是什么？"
qapair = generate_qapair(question)
print(qapair)

NameError: name 'Document' is not defined

In [4]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
import json
import os

# 假设 data 是从 JSON 文件加载的原始数据
data = [{'question': '什么是量子计算?', 'answer': '量子计算是一种利用量子力学原理进行高速计算的技术。'}, 
        {'question': 'Python是什么?', 'answer': 'Python是一种高级编程语言，具有简单易学和功能强大的特点。'}, 
        {'question': '大数据是什么?', 'answer': '大数据指的是无法用常规软件工具在合理时间内捕捉、管理和处理的大量数据。'}, 
        {'question': '机器学习是什么?', 'answer': '机器学习是人工智能的一个分支，它使计算机能够通过数据学习并做出预测或决策。'}]

# 将原始数据转换为 Document 对象列表
docs = [Document(page_content=f"{item['question']}\n{item['answer']}") for item in data]

# 使用 CharacterTextSplitter 分割文档
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

# 打印分割后的文档数量
print(f"Number of documents after splitting: {len(split_docs)}")

# 打印分割后的文档内容
for doc in split_docs:
    print(f"Document content: {doc.page_content}")
    
# 将问答对保存为JSON文件
file_path = 'D:\大一作业\实用机器学习\实训下午\qa_pairs.json'
with open(file_path, 'w') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

file_path
# 1. 知识库路径
knowledge_base_path = 'D:\\大一作业\\实用机器学习\\实训下午\\qa_pairs.json'

# 设置环境变量
os.environ["OPENAI_API_KEY"] = "f28f99afec54ba87aef362f6a6407d3b.kWHsoEEitX2e2Ro5"

# 2. 使用正确的编码读取 JSON 文件
with open(knowledge_base_path, 'r', encoding='GB2312') as file:
    data = json.load(file)

# 3. 确保每个条目都有 page_content，并且是字符串类型
documents = []
for entry in data:
    if 'page_content' in entry and isinstance(entry['page_content'], str):
        documents.append(Document(page_content=entry['page_content']))
    else:
        print(f"Invalid entry: {entry}")

# 将原始数据转换为 Document 对象列表
docs = [Document(page_content=f"{item['question']}\n{item['answer']}") for item in data]

# 使用 CharacterTextSplitter 分割文档
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)


# 5. 创建向量存储
# 使用 OpenAIEmbeddings 来嵌入文档
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# 6. 创建检索问答链
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever())



# 7. 生成问答对
def generate_qa_pair(question: str) -> dict:
    answer = qa.run(question)
    return {"question": question, "answer": answer}

# 示例：生成问答对
question = "LangChain 是什么？"
qa_pair = generate_qa_pair(question)
print(qa_pair)

Number of documents after splitting: 4
Document content: 什么是量子计算?
量子计算是一种利用量子力学原理进行高速计算的技术。
Document content: Python是什么?
Python是一种高级编程语言，具有简单易学和功能强大的特点。
Document content: 大数据是什么?
大数据指的是无法用常规软件工具在合理时间内捕捉、管理和处理的大量数据。
Document content: 机器学习是什么?
机器学习是人工智能的一个分支，它使计算机能够通过数据学习并做出预测或决策。
Invalid entry: {'question': '什么是量子计算?', 'answer': '量子计算是一种利用量子力学原理进行高速计算的技术。'}
Invalid entry: {'question': 'Python是什么?', 'answer': 'Python是一种高级编程语言，具有简单易学和功能强大的特点。'}
Invalid entry: {'question': '大数据是什么?', 'answer': '大数据指的是无法用常规软件工具在合理时间内捕捉、管理和处理的大量数据。'}
Invalid entry: {'question': '机器学习是什么?', 'answer': '机器学习是人工智能的一个分支，它使计算机能够通过数据学习并做出预测或决策。'}


<ipython-input-4-bcbf7ce7738d>:64: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


APIConnectionError: Connection error.

In [5]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-openai
ERROR: No matching distribution found for langchain-openai
